In [1]:
import time
import sklearn
import numpy as np
import pandas as pd

from sklearn.utils import resample

import sys
sys.path.append("..")
from baggingPU import BaggingClassifierPU
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    columnwidth = max([len(x) for x in labels]) + 4
    empty_cell = " " * columnwidth
    print("    " + empty_cell, end=' ')
    for label in labels:
        print("%{0}s".format(columnwidth) % 'pred_' + label, end=" ")
    print()

    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % 'true_' + label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            if cell:
                print(cell, end=" ")
        print()

# import data

In [18]:
#df_raw = pd.read_csv('../data/w-dependence.csv')

df_raw = pd.read_csv('../data/1place-dependence.csv')

#df_raw = pd.read_csv('../data/w-related.csv')

df_raw['label'] = df_raw['label'].astype("int")
print(df_raw.label.value_counts())
print('Has null values', df_raw.isnull().values.any())

1    10000
0    10000
Name: label, dtype: int64
Has null values False


In [19]:
df_raw.head(10)

,p1,p2,p3,p4,label
0,1,0,0,0,1
1,0,1,1,0,1
2,1,0,1,0,1
3,0,0,0,1,1
4,0,1,2,0,1
5,1,0,2,0,1
6,0,0,1,1,1
7,0,1,3,0,1
8,1,0,3,0,1
9,0,0,2,1,1


In [20]:
def random_undersampling(tmp_df, TARGET_LABEL):
    df_majority = tmp_df[tmp_df[TARGET_LABEL] == 0]
    df_minority = tmp_df[tmp_df[TARGET_LABEL] == 1]

    df_majority_downsampled = resample(df_majority, 
                                       replace=False,              
                                       n_samples=len(df_minority), 
                                       random_state=None)        

    df_downsampled = pd.concat([df_majority_downsampled, df_minority])

    print("Undersampling complete!")
    print(df_downsampled[TARGET_LABEL].value_counts())
    return df_downsampled

In [21]:
df_downsampled = random_undersampling(df_raw, 'label')
df_downsampled = df_downsampled.sample(frac=1) 
df_downsampled = df_downsampled.reset_index() 
df_downsampled = df_downsampled.drop(columns=['index']) 

Undersampling complete!
1    10000
0    10000
Name: label, dtype: int64


In [22]:
df_downsampled.head(10)

,p1,p2,p3,p4,label
0,1,1,3440,1,0
1,1,1,8939,1,0
2,596,439,214,360,0
3,0,0,858,1,1
4,0,1,12435,1,0
5,1,0,3124,0,1
6,114,719,642,141,0
7,0,1,1147,0,1
8,1,0,2161,0,1
9,0,0,1084,1,1


In [23]:
x_data = df_raw.iloc[:,:-1]
y_data = df_raw.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

x_train_input = pd.concat([x_train,y_train],axis=1)
x_test_input = pd.concat([x_test,y_test],axis=1)

print(len(x_train_input))
print(len(x_test_input))

print(x_train_input.label.value_counts())
print('Has null values', x_train_input.isnull().values.any())

print(x_test_input.label.value_counts())
print('Has null values', x_test_input.isnull().values.any())

16000
4000
1    8077
0    7923
Name: label, dtype: int64
Has null values False
0    2077
1    1923
Name: label, dtype: int64
Has null values False


In [24]:
x_train_input.head(10)

,p1,p2,p3,p4,label
8118,0,0,2705,1,1
10171,815,965,429,777,0
9399,0,0,3132,1,1
12669,241,761,380,957,0
18809,0,0,10244,0,0
13869,205,120,661,714,0
3320,1,0,1107,0,1
14689,841,618,913,610,0
13087,615,969,311,268,0
15992,0,1,1274,1,0


In [25]:
df = x_train_input.copy()

NON_LBL = [c for c in df.columns if c != 'label']
X = df[NON_LBL]
y = df['label']

y_orig = y.copy()

hidden_size = 5000
y.loc[
    np.random.choice(
        y[y == 1].index, 
        replace = False, 
        size = hidden_size
    )
] = 0

In [26]:
pd.Series(y).value_counts()

0    12923
1     3077
Name: label, dtype: int64

In [27]:
print('- %d samples and %d features' % (X.shape))
print('- %d positive out of %d total before hiding labels' % (sum(df_downsampled.label), len(df_downsampled.label)))
print('- %d positive out of %d total after hiding labels' % (sum(y), len(y)))

- 16000 samples and 4 features
- 10000 positive out of 20000 total before hiding labels
- 3077 positive out of 16000 total after hiding labels


# Trainning directly

In [28]:
print('Training Logistics Regression model ...')

model = LogisticRegression()

model.fit(X, y)

print('Done')

Training Logistics Regression model ...
Done


In [29]:
print('---- {} ----'.format('Logistics Regression'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, model.predict(X)))
print('Recall: ', recall_score(y_orig, model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
print('f1_score: ', f1_score(y_orig, model.predict(X)))

---- Logistics Regression ----
                        pred_negative        pred_positive 
           true_negative       7923.0          0.0 
           true_positive       8077.0          0.0 
None

Precision:  0.0
Recall:  0.0
Accuracy:  0.4951875
f1_score:  0.0


# Training by bagging

In [30]:
print('Training bagging classifier...')

pu_start = time.perf_counter()
model = BaggingClassifierPU(LogisticRegression(),
                         n_estimators = 50, 
                         n_jobs = -1, 
                         max_samples = sum(y) 
                        )
model.fit(X, y)
pu_end = time.perf_counter()
print('Done!')
print('Time:', pu_end - pu_start)

Training bagging classifier...
Done!
Time: 0.9016118300060043


In [31]:
#train data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_orig, model.predict(X)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_orig, model.predict(X)))
print('Recall: ', recall_score(y_orig, model.predict(X)))
print('Accuracy: ', accuracy_score(y_orig, model.predict(X)))
print('f1_score: ', f1_score(y_orig, model.predict(X)))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       7584.0        339.0 
           true_positive          0.0       8077.0 
None

Precision:  0.9597195817490495
Recall:  1.0
Accuracy:  0.9788125
f1_score:  0.979445825501728


In [ ]:
#print wrong predictions
y_pre = model.predict(X)
y_orig_index = y_orig.index.tolist()

FN_index = []
FT_index = []

for i in range(len(y_orig)):
    if y_orig.iloc[i] == 1 and y_pre[i] == 0 :
        FN_index.append(y_orig_index[i])
    if y_orig.iloc[i] == 0 and y_pre[i] == 1 :
        FT_index.append(y_orig_index[i])
        
print("False Negtive:")
print(X.loc[FN_index])
print("False Positive:")
print(X.loc[FT_index])

In [32]:
#test data
print('---- {} ----'.format('PU Bagging'))
print(print_cm(sklearn.metrics.confusion_matrix(y_test, model.predict(x_test)), labels=['negative', 'positive']))
print('')
print('Precision: ', precision_score(y_test, model.predict(x_test)))
print('Recall: ', recall_score(y_test, model.predict(x_test)))
print('Accuracy: ', accuracy_score(y_test, model.predict(x_test)))
print('f1_score: ', f1_score(y_test, model.predict(x_test)))

---- PU Bagging ----
                        pred_negative        pred_positive 
           true_negative       1995.0         82.0 
           true_positive          0.0       1923.0 
None

Precision:  0.9591022443890275
Recall:  1.0
Accuracy:  0.9795
f1_score:  0.9791242362525459


In [ ]:
#print wrong predictions
y_test_pre = model.predict(x_test)
y_test_index = y_test.index.tolist()

FN_test_index = []
FT_test_index = []

for i in range(len(y_test)):
    if y_test.iloc[i] == 1 and y_test_pre[i] == 0 :
        FN_test_index.append(y_test_index[i])
    if y_test.iloc[i] == 0 and y_test_pre[i] == 1 :
        FT_test_index.append(y_test_index[i])

print("False Negtive:")
print(x_test.loc[FN_test_index])
print("False Positive:")
print(x_test.loc[FT_test_index])